# Preamble

In [ ]:
import pandas as pd

from sklearn.model_selection import train_test_split

from plot_utils import visualize_tree, plot_results


In [ ]:
titanic = pd.read_csv('Titanic.csv')
titanic

# Ready datasets

First clean up the data a little

In [ ]:
#one-hot encode categorical features
male_column = pd.get_dummies(titanic["Sex"])[['male']]
embark_columns = pd.get_dummies(titanic["Embarked"])

#replace categorical features with new features
titanic = pd.concat([titanic, male_column, embark_columns], axis='columns').drop(['Sex', 'Embarked'], axis='columns')

#drop non-categorical text features
titanic = titanic.drop(['Name', 'Ticket', 'Cabin'], axis='columns')

#drop data with NaN values
titanic = titanic.dropna()

titanic

Train and test sets

In [ ]:
y = titanic['Survived']
X = titanic.drop('Survived', axis='columns')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=504)


# Training a decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier(random_state=504)
tree_clf.fit(X_train, y_train)

In [ ]:
titanic

<font size = 6>What could possibly go wrong?</font>

In [ ]:
tree_clf.score(X_train, y_train)

Looks like overfitting. How about the test?

In [ ]:
tree_clf.score(X_test, y_test)

What happened?

In [ ]:
visualize_tree(tree_clf, X_train.columns, ["Died", "Survived"])

PassengerId should be irrelevant, but it allows the decision tree to turn into a full binary search tree. Let's drop it.

Also, Fare seems correlated to Pclass. Let's drop that, too.

In [ ]:
X, X_train, X_test = X.drop(['PassengerId', 'Fare'], axis='columns'), X_train.drop("PassengerId", axis='columns'), X_test.drop("PassengerId", axis='columns')

tree_clf.fit(X_train, y_train)
tree_clf.score(X_train, y_train), tree_clf.score(X_test, y_test)

In [ ]:
visualize_tree(tree_clf, X_train.columns, ["Died", "Survived"])

Unconstrained decision trees tend to overfit, because they end up close to full binary search trees. This is also known as _pre-pruning_
We can constrain decision trees in several ways. Let's look at three important hyper-parameters.
* max_depth - puts a limit on how deep the tree can get. *Note*: The tree must be deep enough to accomodate all classes in the classification: 2^depth >= classes
* min_samples_split - forces a minimum of samples in a node before it gets split into subnodes
* max_features - forces the algorithm to only consider a certain number of features per node

### max_depth

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=504, max_depth=3)
tree_clf.fit(X_train, y_train)

tree_clf.score(X_train, y_train), tree_clf.score(X_test, y_test)

In [ ]:
visualize_tree(tree_clf, X_train.columns, ["Died", "Survived"])

This decision tree is much less likely to overfit, because it is forced to generalized more. Of course, if the tree is too shallow, it will underfit.

Let's see how under- and overfitting depends on tree depth.

In [ ]:
features = range(1, 11)

classifiers = [DecisionTreeClassifier(random_state=504, max_depth=d) for d in features]
for clf in classifiers: clf.fit(X_train, y_train)

train_scores = [clf.score(X_train, y_train) for clf in classifiers]
test_scores = [clf.score(X_test, y_test) for clf in classifiers]
plot_results(train_scores, test_scores, train_label="Train accuracy", test_label="Test accuracy", xlabel="depth", ylabel="accuracy")

This illustrates quite clearly how dangerous overfitting is for decision trees. Anyway, looks like a depth of 3 is the sweet spot.

### min_samples_split
Another way of constraining the tree is to be more reluctant to split. For instance, don't split further if there are no more than 10 samples in a leaf.

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=504, min_samples_split=10)
tree_clf.fit(X_train, y_train)

tree_clf.score(X_train, y_train), tree_clf.score(X_test, y_test)

In [ ]:
visualize_tree(tree_clf, X_train.columns, ["Died", "Survived"])

Let's see how the tree performs on different minimal node sizes.

In [ ]:
sizes = range(2, 31)

classifiers = [DecisionTreeClassifier(random_state=504, min_samples_split=s) for s in sizes]
for clf in classifiers: clf.fit(X_train, y_train)

train_scores = [clf.score(X_train, y_train) for clf in classifiers]
test_scores = [clf.score(X_test, y_test) for clf in classifiers]
plot_results(train_scores, test_scores, train_label="Train accuracy", test_label="Test accuracy", xlabel="min split size", ylabel="accuracy")

In [ ]:
visualize_tree(tree_clf, X_train.columns, ["Died", "Survived"]).view()

A shallow tree will usually have rather large samples in the leaves, so it doesn't make much sense to mix max_depth and min_samples_split.

An alternative way of telling the algorithm "only split if it's worth it" is to forbid splitting a node unless the improvement is big enough: min_impurity_decrease (see visualization)

### max_features

This is an odd duck. At each node the algorithm chooses a random subset of the features, which are used to evaluate the best split. It does *not* mean 'only split on the best n features', it means 'split on n *random* features'.

This makes for a faster algorithm if we have many features, but it also introduces random bias.

In [ ]:
features = range(1, 10)

classifiers = [DecisionTreeClassifier(random_state=504, max_features=f, max_depth=3) for f in features]
for clf in classifiers: clf.fit(X_train, y_train)

train_scores = [clf.score(X_train, y_train) for clf in classifiers]
test_scores = [clf.score(X_test, y_test) for clf in classifiers]
plot_results(train_scores, test_scores, train_label="Train accuracy", test_label="Test accuracy", xlabel="min split size", ylabel="accuracy")

Clearly, max_features should be 4. 

**What am I doing wrong here?**

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=504, max_features=4, max_depth=3)
tree_clf.fit(X_train, y_train)

tree_clf.score(X_train, y_train), tree_clf.score(X_test, y_test)

While this looks nice and not at all overfitting, we are tuning the hyperparameters on the test set. Now we need another set for validation.

# Cross Validation

Let us instead search for the best solution using cross validation. The nice thing about cross validation is that it doesn't use the test set for validation but part of the train set. So we can still test for overfitting.

In [ ]:
from sklearn.model_selection import GridSearchCV
depths = range(1, 11)
sizes = range(2, 21)
features = range(1, 9)
params = {'min_samples_split': sizes, 
          'max_depth': depths, 
          'max_features': features}

gsc = GridSearchCV(DecisionTreeClassifier(random_state=504), params, n_jobs=16)
gsc.fit(X_train, y_train)
gsc.best_estimator_

In [ ]:
gsc.score(X_train, y_train), gsc.score(X_test, y_test)

This seems good, but try it with some different random seeds. There is a lot of noise here, and it's probably best to keep it simple. 

In [ ]:
importances = zip(X_train.keys(), gsc.best_estimator_.feature_importances_) #pairs up feature names with performance score
sorted(importances, key=lambda p: -p[1])